In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import re
import numpy as np 
from random import shuffle
from time import time 



Using TensorFlow backend.


# 1. Supervised Parameter Estimation for HMMs (15 points)

## 1.1 Implementation

In [2]:
train = open('en_ewt.train',encoding="utf8").read()
dev = open('en_ewt.dev',encoding="utf8").read()
corpus = list(map(lambda x: list(map(lambda y: y.split("\t"), x.split("\n"))),train.split("\n\n")))
corpus = corpus[:-1]

word_index = {}
index_word = {}
word_index['<s>']= 0 
word_index['</s>']= 1
index_word[0] = '<s>'
index_word[1] = '</s>'

tag_index = {}
index_tag = {}
tag_index['<s>']= 0 
tag_index['</s>']= 1
index_tag[0] = '<s>'
index_tag[1] = '</s>'

m = 2 
k = 2
for sent in corpus:
    for w, t in sent:
        if t not in tag_index:
            tag_index[t] = k 
            index_tag[k] = t
            k+= 1 
        if w not in word_index:
            word_index[w] = m 
            index_word[m] = w
            m += 1 
        
            
num_words = len(word_index)
e_mat = np.zeros((52,num_words))
t_mat = np.zeros((52,52))

for sent in corpus:
    prev = '<s>'
    for w, t in sent:
        i_prev,i_cur,i_word = tag_index[prev],tag_index[t],word_index[w]
        t_mat[i_prev][i_cur] +=  1
        e_mat[i_cur][i_word] += 1
        
        prev = t 
    
    t = '</s>'
    i_prev,i_cur = tag_index[prev],tag_index[t]
    t_mat[i_prev][i_cur] = t_mat[i_prev][i_cur]  + 1
    
l_t = 0.1 
l_e = 0.001

t_mat  = t_mat + l_t
e_mat = e_mat + l_e 

t_mat[0][1] = 0 
t_mat[:,0] = 0


row_sums = t_mat.sum(axis=1)
t_mat = t_mat / row_sums[:, np.newaxis]
t_mat[1] = 0

row_sums = e_mat.sum(axis=1)
e_mat = e_mat / row_sums[:, np.newaxis]
e_mat[0:2] = 0 

## 1.2 Verify implementation correctness

In [14]:
p = list(zip(t_mat[0][t_mat[0].argsort()[-5:]],t_mat[0].argsort()[-5:])) 

print("----5 tags with the highest probability of following <s>----")

for score,id_ in p[::-1]:
    print("probability: ",end="")
    print(score,end=",")
    print("tag: {}".format(index_tag[id_]))

print("----10 most probable words emitted by the adjective tag('JJ')----")

p = list(zip(e_mat[tag_index['JJ']][e_mat[tag_index['JJ']].argsort()[-10:]],
            e_mat[tag_index['JJ']].argsort()[-10:]))
for score,id_ in p[::-1]:
    print("Probability: ",end="")
    print(score,end=",")
    print("word: {}".format(index_word[id_]))


----5 tags with the highest probability of following <s>----
probability: 0.22450589735415996,tag: PRP
probability: 0.11596270321963656,tag: NNP
probability: 0.11237647433853996,tag: DT
probability: 0.0781080650302837,tag: IN
probability: 0.06567580490914886,tag: RB
----10 most probable words emitted by the adjective tag('JJ')----
Probability: 0.023068966829187507,word: other
Probability: 0.021605642303920113,word: good
Probability: 0.016785279161862805,word: new
Probability: 0.013686474284825965,word: many
Probability: 0.013686474284825965,word: great
Probability: 0.011362370627048334,word: same
Probability: 0.009554734448776844,word: sure
Probability: 0.009210422795772751,word: last
Probability: 0.00895218905601968,word: few
Probability: 0.00843572157651354,word: little


# 2. Preliminaries for Inference with HMMs (5 Points)

## 2.1 Log-probability calculation (2 points)

In [100]:

def preprocess(data):
    corpus = list(map(lambda x: list(map(lambda y: y.split("\t"), x.split("\n"))),data.split("\n\n")))
    sent = []
    tags = []
    for i in corpus: 
        length = 0
        ss = np.zeros(len(i))
        tt = np.zeros(len(i))
        for j in range(len(i)):
            if len(i[j]) ==  2:
                ss[j] = word_index[i[j][0]]
                tt[j] = tag_index[i[j][1]]
                length += 1
                
        if length != 0:
            sent.append(ss.astype(np.int))
            tags.append(tt.astype(np.int))
    return sent,tags

def log_prob(x,y):
    res = 0
    for i in range(len(x)-1):
        i_word,i_tag = x[i],y[i]
        next_tag = y[i+1]
        res += np.log(e_mat[i_tag][i_word]) + np.log(t_mat[i_tag][next_tag])
    res += np.log(t_mat[0][y[0]]) 
    res += np.log(t_mat[y[-1]][1]) + + np.log(e_mat[y[-1]][x[-1]])   
    return res
    

In [103]:
total_prob  = 0 
dev_x, dev_y = preprocess(dev)
for i in range(len(dev_x)):
    total_prob += log_prob(dev_x[i],dev_y[i])
print(total_prob)

-169936.4097817666


## 2.2 Local predictor baseline (3 points): 

In [169]:
from time import time
def local_predict(sent):
    tag = e_mat[:,sent].argmax(axis=0)
    return tag

def evaluate(predict,truth):
    total = 0
    correct = 0
    for i in range(len(predict)):
        correct += (predict[i] == truth[i]).sum()
        total += truth[i].shape[0] 
    return total,correct

s = time()
prediction = []
for i in range(2002):
    prediction.append(local_predict(dev_x[i]))
    
print("time_elapse in sec: ",time()-s)

total,correct = evaluate(prediction,dev_y)

print("accuracy: ", correct/total)

total_prob=0
for i in range(len(dev_x)):
    total_prob += log_prob(dev_x[i],prediction[i])
print("log prob: ", total_prob)


time_elapse in sec:  0.03206777572631836
accuracy:  0.7988866799204771
log prob:  -186408.17332565217


# 3. Greedy Left-to-Right Algorithms (10 points)


In [186]:
from time import time
def greedy_leftright(sent):
    tag = np.zeros(sent.shape[0]).astype(np.int)
    prev = 0    
    for i in range(sent.shape[0]-1):
        
        tag[i] = int((e_mat[:,sent[i]]* t_mat[prev]).argmax())
        prev = tag[i]
    tag[-1] = (e_mat[:,sent[-1]]* t_mat[prev] * t_mat[:,1]).argmax()
    
    return tag

def evaluate(predict,truth):
    total = 0
    correct = 0
    for i in range(len(predict)):
        correct += (predict[i] == truth[i]).sum()
        total += truth[i].shape[0] 
    return total,correct

s = time()
prediction = []
for i in range(2002):
    prediction.append(greedy_leftright(dev_x[i]))
    
print("time_elapse in sec: ",time()-s)

total,correct = evaluate(prediction,dev_y)

print("accuracy: ", correct/total)

total_prob=0
for i in range(len(dev_x)):
    total_prob += log_prob(dev_x[i],prediction[i])
print("log prob: ", total_prob)


time_elapse in sec:  0.3826887607574463
accuracy:  0.8740357852882704
log prob:  -166188.90291524684


# 4. Greedy Right-to-Left Algorithms (10 points

In [204]:
from time import time
def greedy_rightleft(sent):
    tag = np.zeros(sent.shape[0]).astype(np.int)
    next_ = 1   
    for i in range(1,sent.shape[0]):        
        tag[-i] = (e_mat[:,sent[-i]]* t_mat[:,next_]).argmax()
        next_ = tag[-i]
    tag[0] = (e_mat[:,sent[0]]* t_mat[:,next_] * t_mat[0]).argmax()
    
    return tag

def evaluate(predict,truth):
    total = 0
    correct = 0
    for i in range(len(predict)):
        correct += (predict[i] == truth[i]).sum()
        total += truth[i].shape[0] 
    return total,correct

s = time()
prediction = []
for i in range(2002):
    prediction.append(greedy_rightleft(dev_x[i]))
    
print("time_elapse in sec: ",time()-s)

total,correct = evaluate(prediction,dev_y)

print("accuracy: ", correct/total)

total_prob=0
for i in range(len(dev_x)):
    total_prob += log_prob(dev_x[i],prediction[i])
print("log prob: ", total_prob)




time_elapse in sec:  0.21828842163085938
accuracy:  0.8001590457256461
log prob:  -176754.99936669567


Discuss the results. In your discussion, consider the following points. What do you
notice about the performance (accuracy and log-probability) of this algorithm compared to the
local and left-to-right algorithms? Why do you think you see these trends? What changes might
you make to improve the accuracy of greedy right-to-left algorithms?



Answer: 

Interestingly, we see log prob shows LF (left-to-right) has higher log prob than RF (right-to-left), and RF has higher log prob than LC(local). However, the accuracy doesn't quite match with this difference, because RF only outforms LC by a very small margin while there's huge improvement from LF to RF. 

I think this is because of model error. One aspect is that the model  might predict the last words badly, so later predictions of RF which are contigent upon the previous predictions also become bad. (While LF can better predict the left starting word). This may be because words on the left of a sentence carry more info than on the right of a sentence.

Furthermore, when we train the model, we learn the probability from left to right, but the conditional probability of P(A|B) is different from P(B|A), so when we use the model to predict from right to left (say from B to A), we're actually using P(A|B), which might not give us the most possible left word given the right word B. 

So one change might be to train the model by learning the conditional probability of the left word given the right word (i.e. right to left instead of left to right), and then run the right to left greedy algorithm based on this. 

# 5. Exact Inference with Viterbi (20 points)

In [236]:
from time import time

def viterbi(sent):
    dp = np.zeros(t_mat.shape[0])
    dp = e_mat[:,sent[0]] * t_mat[0]
    history = np.zeros((t_mat.shape[0],len(sent)-1))
    tag = np.zeros(len(sent)).astype(np.int)
    
    for i in range(1,sent.shape[0]):  
        temp = dp*t_mat.T
        history[:,i-1] = temp.argmax(axis=1)
        dp = e_mat[:,sent[i]] * (temp.max(axis=1)) 
        
    end = (t_mat[:,1] * dp).argmax()
    tag[-1] = end
    for i in range(1,sent.shape[0]):
        tag[-i-1] = history[end][-i] 
        end = tag[-i-1]
    
    return tag

def evaluate(predict,truth):
    total = 0
    correct = 0
    for i in range(len(predict)):
        correct += (predict[i] == truth[i]).sum()
        total += truth[i].shape[0] 
    return total,correct

s = time()
prediction = []
for i in range(2002):
    prediction.append(viterbi(dev_x[i]))
    
print("time_elapse in sec: ",time()-s)

total,correct = evaluate(prediction,dev_y)

print("accuracy: ", correct/total)

total_prob=0
for i in range(len(dev_x)):
    total_prob += log_prob(dev_x[i],prediction[i])
print("log prob: ", total_prob)


##(b) see latex response

time_elapse in sec:  1.3345210552215576
accuracy:  0.8941153081510934
log prob:  -163206.91122073226


# 6.  Extra Credit: Beam Search for HMMs (5 points)